In [1]:
import pandas as pd
# set ramdom seed for reproducibility
import numpy as np
np.random.seed(49)

In [2]:
# Load the data from csv
df = pd.read_csv("new_fakeaudio_dir.csv")
df.head()

,vid_dir,text_dir,vid_output_dir
0,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-FakeAudio/African/m...
1,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-FakeAudio/African/m...
2,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-FakeAudio/African/m...
3,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-FakeAudio/African/m...
4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,FakeAVCeleb_v1.2/RealVideo-FakeAudio/African/m...


In [3]:
# need to update path in front of the vid_dir column
path = "/"
df['vid_dir'] = path + df['vid_dir']

# drop the second and third columns
df = df.drop(columns=['text_dir', 'vid_output_dir'])

In [4]:
# split vid_dir into vid_dirs and vid_name by "/"
df['category'] = df['vid_dir'].apply(lambda x: x.split("/")[4])
df['racial'] = df['vid_dir'].apply(lambda x: x.split("/")[5])
df['gender'] = df['vid_dir'].apply(lambda x: x.split("/")[6])
df['ID'] = df['vid_dir'].apply(lambda x: "/".join(x.split("/")[7:]))
# df['vid_name'] = df['vid_dir'].apply(lambda x: x.split("/")[8])

df.head()

,vid_dir,category,racial,gender,ID
0,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00076/00109.mp4
1,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00166/00010.mp4
2,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00173/00118.mp4
3,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00366/00118.mp4
4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00391/00052.mp4


In [5]:
# group by the same category, racial, and gender
df_grouped = df.groupby(['category', 'racial', 'gender'])

def assign_samples(group):
    # Ensure the group is shuffled; you can remove `.sample(frac=1)` if not needed
    shuffled_group = group.sample(frac=1).reset_index(drop=True)
    
    # Assign each sample with the next one
    assigned = shuffled_group.shift(-1)
    
    # For the last sample, circle back to the first sample
    if len(assigned) > 0:
        assigned.iloc[-1] = shuffled_group.iloc[0]
    
    return assigned

# Apply the function to each group
assigned_df = df_grouped.apply(assign_samples).reset_index(drop=True)

assigned_df.head()

,vid_dir,category,racial,gender,ID
0,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id01637/00002.mp4
1,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id01972/00078.mp4
2,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id01076/00005.mp4
3,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00987/00160.mp4
4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id01207/00320.mp4


In [6]:
df['paired_vid_dir'] = assigned_df['vid_dir']
df['paired_ID'] = assigned_df['ID']
df.head()

,vid_dir,category,racial,gender,ID,paired_vid_dir,paired_ID
0,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00076/00109.mp4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,id01637/00002.mp4
1,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00166/00010.mp4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,id01972/00078.mp4
2,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00173/00118.mp4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,id01076/00005.mp4
3,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00366/00118.mp4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,id00987/00160.mp4
4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,RealVideo-RealAudio,African,men,id00391/00052.mp4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,id01207/00320.mp4


In [7]:
# assert that vid_dir and paired_vid_dir are not the same
assert (df['vid_dir'] != df['paired_vid_dir']).all()

# return the idx if the vid_dir and paired_vid_dir are the same
idx = df[df['vid_dir'] == df['paired_vid_dir']].index

In [8]:
(df['vid_dir'] == df['paired_vid_dir']).sum()

0

In [9]:
# find the index of 1 in (df['vid_dir'] == df['paired_vid_dir'])
idx = df[df['vid_dir'] == df['paired_vid_dir']].index
idx

Index([], dtype='int64')

In [10]:
# drop columns of "category", "racial", "gender", "ID"
df = df.drop(columns=["category", "racial", "gender", "ID"])
df["output_dir"] = df["paired_vid_dir"].apply(lambda x: "/".join(x.split("/")[:7]) + "/") 
# concat str of output_dir and paired_ID
df["output_dir"] = df["output_dir"] + df["paired_ID"]
# change ".mp4" to ".wav"
df["vid_dir"] = df["vid_dir"].apply(lambda x: x.replace(".mp4", ".wav"))
df["paired_vid_dir"] = df["paired_vid_dir"].apply(lambda x: x.replace(".mp4", ".wav"))
df["output_dir"] = df["output_dir"].apply(lambda x: x.replace(".mp4", "_fake_sync.wav"))
# drop paired_ID
df = df.drop(columns=["paired_ID"])
df["output_dir"] = df["output_dir"].apply(lambda x: x.replace("RealAudio", "FakeAudio"))
df.head()

,vid_dir,paired_vid_dir,output_dir
0,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Fake...
1,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Fake...
2,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Fake...
3,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Fake...
4,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Real...,/data/kyungbok/FakeAVCeleb_v1.2/RealVideo-Fake...


In [11]:
# save the dataframe to csv
df.to_csv("paired_fakeaudio_dir.csv", index=False)